# Diferenciation between Azure SDK  and Cloud Data Connector

In [ ]:
from pathlib import Path
import os
import getpass
import shutil
from urllib.request import urlretrieve

## Input details
Required data for both cases requires a connection string, container name and blob file.

In [ ]:
connection_string = getpass.getpass("Insert Azure blob client connection string")
container_name = input("Container Name. (default: data-connector)")
if not container_name:
    container_name = "data-connector"
file_path = input("Upload file (default: .\samples\credit_card_clients.xls)")
if not file_path:
    dataset = "https://azuremlexamples.blob.core.windows.net/datasets/credit_card/default_of_credit_card_clients.csv"
    file_name = "credit_card_clients.xls"
    file_name_cdc = "credit_card_clients2.xls"
    urlretrieve(dataset, file_name)
    file_path = os.path.join(Path().resolve(), file_name)
    shutil.copy2(file_path, file_name_cdc)
else:
    file_name = Path(file_path).name

print(f"Container: {container_name}, file name: {file_name}")


## Azure SDK

### Import Azure BlobServiceClient and Exception

In [ ]:
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError

### Connecting BlobServiceClient

In [ ]:
blob_service_client: BlobServiceClient = BlobServiceClient.from_connection_string(conn_str=connection_string)
container_client = blob_service_client.get_container_client(container_name)

### Upload File

In [ ]:
if os.path.exists(file_path):
    try:
        print(f"file_name: {file_name}")
        print(f"file path: {file_path}")
        blob_service_client = BlobServiceClient.from_connection_string(
            connection_string
        )

        container_client = blob_service_client.get_container_client(container_name)
        with open(file_path, 'rb') as data:
            container_client.upload_blob(name=file_name ,data=data)
    except ResourceExistsError as ree:
        print(ree)
else:
    print(f"File missed:  {file_path}")



### Download File

In [ ]:
storage_stream_downloader = container_client.download_blob(
    file_name
)

# Cloud Data Connector


### Import azure Cloud Data Connector  

In [ ]:
from cloud_data_connector.azure import Connector, Uploader, Downloader, connect

### Upload File

In [ ]:
connector: BlobServiceClient = connect(connection_string)
# Upload file
uploader: Uploader = Uploader(connector)
uploader.upload(source_path=file_name_cdc, blob_container_name=container_name)
print(type(connector))

### Download File

#### Connect Cloud Data Connector to Azure

In [ ]:
downloader: Downloader = Downloader(
   connector
)

#### Dowload File

In [ ]:

downloader.download(
    donwload_obj=container_name,
    data_file=file_name_cdc,
    destiny=file_name_cdc+"dc_downloaded"
)
